# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json 

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [5]:
file=pd.read_csv('weather_data.csv')

file

,Unnamed: 0,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,0,Kumul,45,CN,1611605200,73,42.8000,93.4500,15.64,1.36
1,1,Hong Kong,1,HK,1611605112,86,22.2855,114.1577,64.99,1.99
2,2,Hobart,75,AU,1611605201,94,-42.8794,147.3294,62.01,6.91
3,3,Hamilton,90,US,1611604715,86,39.1834,-84.5333,33.80,9.22
4,4,Butaritari,94,KI,1611605201,79,3.0707,172.7902,79.14,10.00
...,...,...,...,...,...,...,...,...,...,...
553,553,Gayny,100,RU,1611605305,97,60.3071,54.3241,35.29,7.92
554,554,Cedral,21,MX,1611605305,17,23.8000,-100.7333,77.86,21.56
555,555,Astoria,40,US,1611605306,87,46.1879,-123.8313,39.20,4.45
556,556,Posse,68,BR,1611605306,53,-14.0931,-46.3694,82.78,2.30


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [6]:
#configure gmaps with API key
gmaps.configure(api_key=g_key)


In [8]:
#store lat and long as locations and humidity as weight 
locations=file[['Lat','Lng']]
humidity=file['Humidity'].astype(float)

In [10]:
#plot heatmap 
fig=gmaps.figure(center=[0,0],zoom_level=2)

#create and add heat layer 
heat_layer=gmaps.heatmap_layer(locations,weights=humidity,dissipating=False,max_intensity=100,point_radius=4)

fig.add_layer(heat_layer)
plt.savefig("Vacationheatmap.png")
fig

Figure(layout=FigureLayout(height='420px'))

<Figure size 432x288 with 0 Axes>

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [15]:
# A max temperature lower than 80 degrees but higher than 70.

temp=file.loc[file['Max Temp']<80,:]
temp_df=file.loc[file['Max Temp']>70,:]
temp_df.head()

#Wind speed less than 10 mph.

wind_speed=temp_df.loc[temp_df['Wind Speed']<10,:]
wind_speed

#Zero cloudiness.
ideal_weather=wind_speed.loc[wind_speed['Cloudiness']==0,:]
ideal_weather.head()


,Unnamed: 0,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
92,92,Saint-Pierre,0,RE,1611605219,73,-21.3393,55.4781,78.80,3.44
145,145,Atar,0,MR,1611605229,16,20.5169,-13.0499,73.04,8.70
160,160,Ouadda,0,CF,1611605232,20,8.0777,22.4007,77.58,7.20
165,165,Tahoua,0,NE,1611605233,11,14.8888,5.2692,78.80,4.61
185,185,Umluj,0,SA,1611605236,65,25.0213,37.2685,71.06,7.65


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [22]:
hotel_df=ideal_weather
hotel_df['Hotel Name']= " "
hotel_df

<ipython-input-22-fa8a4c8174b4>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hotel_df['Hotel Name']= " "


,Unnamed: 0,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name
92,92,Saint-Pierre,0,RE,1611605219,73,-21.3393,55.4781,78.80,3.44,
145,145,Atar,0,MR,1611605229,16,20.5169,-13.0499,73.04,8.70,
160,160,Ouadda,0,CF,1611605232,20,8.0777,22.4007,77.58,7.20,
165,165,Tahoua,0,NE,1611605233,11,14.8888,5.2692,78.80,4.61,
185,185,Umluj,0,SA,1611605236,65,25.0213,37.2685,71.06,7.65,
213,213,Saint-Philippe,0,RE,1611605242,73,-21.3585,55.7679,78.80,3.44,
247,247,General Roca,0,AR,1611605249,15,-39.0333,-67.5833,91.40,9.22,
249,249,Idah,0,NG,1611605249,22,7.1065,6.7342,82.67,2.77,
283,283,Tiassalé,0,CI,1611605255,77,5.8983,-4.8283,78.96,5.66,
287,287,Sydney,0,AU,1611605044,83,-33.8679,151.2073,75.20,5.75,


In [25]:
for index, row in hotel_df.iterrows():
    try:
        
        base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

        params = {
        "keyword": "hotel",
        "radius": 5000,
        "key": g_key,
        }


        lat = row['Lat']
        lng = row['Lng']

        params['location'] = f"{lat}, {lng}"

        hotel_data = requests.get(base_url, params=params).json()
        
        hotel_df.loc[index, "Hotel Name"] = hotel_data["results"][0]["name"]
        
    except IndexError:
        
        hotel_df.loc[index, "Hotel Name"] = "NaN"
        
hotel_df

/Users/aishy/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1765: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)


,Unnamed: 0,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name
92,92,Saint-Pierre,0,RE,1611605219,73,-21.3393,55.4781,78.80,3.44,LODGE PALMAE
145,145,Atar,0,MR,1611605229,16,20.5169,-13.0499,73.04,8.70,Odar kanawal
160,160,Ouadda,0,CF,1611605232,20,8.0777,22.4007,77.58,7.20,NaN
165,165,Tahoua,0,NE,1611605233,11,14.8888,5.2692,78.80,4.61,Hotel Etab
185,185,Umluj,0,SA,1611605236,65,25.0213,37.2685,71.06,7.65,HP Red Sea Hotel
213,213,Saint-Philippe,0,RE,1611605242,73,-21.3585,55.7679,78.80,3.44,Les Embruns Du Baril
247,247,General Roca,0,AR,1611605249,15,-39.0333,-67.5833,91.40,9.22,Estepa Hotel
249,249,Idah,0,NG,1611605249,22,7.1065,6.7342,82.67,2.77,Fosla Hotels - Idah
283,283,Tiassalé,0,CI,1611605255,77,5.8983,-4.8283,78.96,5.66,"Hotel ""Pacific Hotel de Tiassale"""
287,287,Sydney,0,AU,1611605044,83,-33.8679,151.2073,75.20,5.75,Sydney Boutique Hotel


In [27]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [29]:
# Add marker layer ontop of heat map


markers = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(markers)
plt.savefig("Vacationheatmaphotels.png")
fig

# Display figure


Figure(layout=FigureLayout(height='420px'))

<Figure size 432x288 with 0 Axes>